# SI 330: Data Manipulation 
## 06 - Enhancing pandas peformance

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

## Learning Objectives

* be able to describe 5 types of Big O notation
* explain why we should be concerned with algorithmic efficiency
* be able to report execution times in Jupyter
* be able to explain profiling output in Jupyter

## Big O Notation

* "order of growth"
* a way to categorize how algorithms behave with increasing data size
* commonly used in Computer Science

https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/

### $O(1)$

In [63]:
def IsFirstElementNull(aList):
    return aList[0] == None

### $O(n)$

In [64]:
def ContainsValue(aList, aValue):
    for e in aList:
        if e == aValue:
            return True
    return False

### $O(n^2$)

In [35]:
def ContainsDuplicates(aList):
    for i in range(len(aList)):
        for j in range(len(aList)):
            if i == j:
                continue
            if aList[i] == aList[j]:
                return True
    return False

### $O(2^n$)

In [36]:
def Fibonacci(n):
    if (n <= 1):
        return n
    return Fibonacci(n - 2) + Fibonacci(n -1 )

### $O(log(n))$

In each iteration, we are halfway closer to finishing the problem.

https://stackoverflow.com/questions/2307283/what-does-olog-n-mean-exactly

This is why, for example, looking up people in a phone book is O(log n). You don't need to check every person in the phone book to find the right one; instead, you can simply divide-and-conquer by looking based on where their name is alphabetically, and in every section you only need to explore a subset of each section before you eventually find someone's phone number.

Of course, a bigger phone book will still take you a longer time, but it won't grow as quickly as the proportional increase in the additional size.

We can expand the phone book example to compare other kinds of operations and their running time. We will assume our phone book has businesses (the "Yellow Pages") which have unique names and people (the "White Pages") which may not have unique names. A phone number is assigned to at most one person or business. We will also assume that it takes constant time to flip to a specific page.

Here are the running times of some operations we might perform on the phone book, from best to worst:

O(1) (best case): Given the page that a business's name is on and the business name, find the phone number.

O(1) (average case): Given the page that a person's name is on and their name, find the phone number.

O(log n): Given a person's name, find the phone number by picking a random point about halfway through the part of the book you haven't searched yet, then checking to see whether the person's name is at that point. Then repeat the process about halfway through the part of the book where the person's name lies. (This is a binary search for a person's name.)

O(n): Find all people whose phone numbers contain the digit "5".

O(n): Given a phone number, find the person or business with that number.

O(n log n): There was a mix-up at the printer's office, and our phone book had all its pages inserted in a random order. Fix the ordering so that it's correct by looking at the first name on each page and then putting that page in the appropriate spot in a new, empty phone book.

For the below examples, we're now at the printer's office. Phone books are waiting to be mailed to each resident or business, and there's a sticker on each phone book identifying where it should be mailed to. Every person or business gets one phone book.

O(n log n): We want to personalize the phone book, so we're going to find each person or business's name in their designated copy, then circle their name in the book and write a short thank-you note for their patronage.

O(n2): A mistake occurred at the office, and every entry in each of the phone books has an extra "0" at the end of the phone number. Take some white-out and remove each zero.

O(n · n!): We're ready to load the phonebooks onto the shipping dock. Unfortunately, the robot that was supposed to load the books has gone haywire: it's putting the books onto the truck in a random order! Even worse, it loads all the books onto the truck, then checks to see if they're in the right order, and if not, it unloads them and starts over. (This is the dreaded bogo sort.)

O(nn): You fix the robot so that it's loading things correctly. The next day, one of your co-workers plays a prank on you and wires the loading dock robot to the automated printing systems. Every time the robot goes to load an original book, the factory printer makes a duplicate run of all the phonebooks! Fortunately, the robot's bug-detection systems are sophisticated enough that the robot doesn't try printing even more copies when it encounters a duplicate book for loading, but it still has to load every original and duplicate book that's been printed.

For more mathematical explanation you can checkout how the time complexity arrives to log n here. https://hackernoon.com/what-does-the-time-complexity-o-log-n-actually-mean-45f94bb5bfbf

## Timing and Profiling in Jupyter

The following code and examples are from [Sofia Heisler's repo](https://github.com/s-heisler/pycon2017-optimizing-pandas.git).
It is available at https://github.com/umsi-data-science/s-heisler-pycon2017-optimizing-pandas

In [37]:
import pandas as pd
import numpy as np
from math import *

### Read in the data

In [40]:
df = pd.read_csv('data/new_york_hotels.csv', encoding='cp1252')

In [41]:
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


In [42]:
df.columns

Index(['ean_hotel_id', 'name', 'address1', 'city', 'state_province',
       'postal_code', 'latitude', 'longitude', 'star_rating', 'high_rate',
       'low_rate'],
      dtype='object')

## Benchmarking example

#### Define the normalization function

In [43]:
def normalize(df, pd_series):
    pd_series = pd_series.astype(float)
    #print(pd_series)
    # Find upper and lower bound for outliers
    avg = np.mean(pd_series)
    sd  = np.std(pd_series)
    #print(avg,sd)
    lower_bound = avg - 2*sd
    upper_bound = avg + 2*sd

    # Collapse in the outliers
    df.loc[pd_series < lower_bound , "high_rate" ] = lower_bound
    df.loc[pd_series > upper_bound , "high_rate" ] = upper_bound
    # Finally, take the log (why add 1?)
    normalized_price = np.log(df["high_rate"].astype(float)+1)
    return normalized_price

In [44]:
normalize(df, df['high_rate'])

0       5.043601
1       5.193012
2       4.906533
3       5.734249
4       5.138090
5       5.247164
6       5.072872
7       5.422612
8       5.074459
9       4.941817
10      4.877789
11      5.688060
12      4.913116
13      5.247866
14      5.164786
15      4.895224
16      5.254627
17      4.266054
18      4.979557
19      5.193401
20      4.323205
21      4.262539
22      4.756517
23      4.941642
24      5.199712
25      5.274537
26      4.565084
27      5.248602
28      4.608963
29      4.204685
          ...   
1601    5.198497
1602    5.171733
1603    5.252430
1604    4.765246
1605    5.013697
1606    4.838581
1607    4.831349
1608    4.798102
1609    4.664397
1610    5.348915
1611    4.269558
1612    3.914272
1613    4.619369
1614    4.330733
1615    4.942531
1616    3.931826
1617    4.992433
1618    4.985728
1619    5.380391
1620    6.019323
1621    5.087535
1622    5.526272
1623    5.220356
1624    5.135974
1625    5.081404
1626    5.599754
1627    4.867534
1628    5.5227

#### Timing the normalization function

In [45]:
%timeit df['high_rate_normalized'] = normalize(df, df['high_rate'])

3.79 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Profiling the normalization function

**NOTE: You only need to conda install -y line_profiler ONCE!**

In [46]:
!conda install -y line_profiler

Solving environment: done

# All requested packages already installed.



In [47]:
    %load_ext line_profiler

In [48]:
%lprun -f normalize df['high_rate_normalized'] =\
        normalize(df, df['high_rate'])

Timer unit: 1e-06 s

Total time: 0.013697 s
File: <ipython-input-43-6e7cab35a1d5>
Function: normalize at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def normalize(df, pd_series):
     2         1        388.0    388.0      2.8      pd_series = pd_series.astype(float)
     3                                               #print(pd_series)
     4                                               # Find upper and lower bound for outliers
     5         1        464.0    464.0      3.4      avg = np.mean(pd_series)
     6         1        168.0    168.0      1.2      sd  = np.std(pd_series)
     7                                               #print(avg,sd)
     8         1          2.0      2.0      0.0      lower_bound = avg - 2*sd
     9         1          1.0      1.0      0.0      upper_bound = avg + 2*sd
    10                                           
    11                                               # Collap

## Haversine definition

https://en.wikipedia.org/wiki/Haversine_formula

In [49]:
def haversine(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

## Iterrows Haversine

In [50]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985,\
                                      row['latitude'], row['longitude']))
df['distance'] = haversine_series

125 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Apply Haversine on rows

### Timing "apply"

In [51]:
%timeit df['distance'] =\
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

61.9 ms ± 2.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Profiling "apply"

In [52]:
# Haversine applied on rows
%lprun -f haversine \
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

Timer unit: 1e-06 s

Total time: 0.042031 s
File: <ipython-input-49-ad000f5aaf6c>
Function: haversine at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def haversine(lat1, lon1, lat2, lon2):
     2      1631       1110.0      0.7      2.6      miles_constant = 3959
     3      1631      11013.0      6.8     26.2      lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
     4      1631       1547.0      0.9      3.7      dlat = lat2 - lat1 
     5      1631        841.0      0.5      2.0      dlon = lon2 - lon1 
     6      1631      18557.0     11.4     44.2      a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
     7      1631       7005.0      4.3     16.7      c = 2 * np.arcsin(np.sqrt(a)) 
     8      1631       1294.0      0.8      3.1      mi = miles_constant * c
     9      1631        664.0      0.4      1.6      return mi

## Vectorized implementation of Haversine applied on Pandas series

#### Timing vectorized implementation

In [53]:
# Vectorized implementation of Haversine applied on Pandas series
%timeit df['distance'] = haversine(40.671, -73.985,\
                                   df['latitude'], df['longitude'])

1.31 ms ± 56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Profiling vectorized implementation

In [54]:
# Vectorized implementation profile
%lprun -f haversine haversine(40.671, -73.985,\
                              df['latitude'], df['longitude'])

Timer unit: 1e-06 s

Total time: 0.003026 s
File: <ipython-input-49-ad000f5aaf6c>
Function: haversine at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def haversine(lat1, lon1, lat2, lon2):
     2         1          1.0      1.0      0.0      miles_constant = 3959
     3         1        228.0    228.0      7.5      lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
     4         1        229.0    229.0      7.6      dlat = lat2 - lat1 
     5         1        178.0    178.0      5.9      dlon = lon2 - lon1 
     6         1       1821.0   1821.0     60.2      a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
     7         1        394.0    394.0     13.0      c = 2 * np.arcsin(np.sqrt(a)) 
     8         1        174.0    174.0      5.8      mi = miles_constant * c
     9         1          1.0      1.0      0.0      return mi

## Vectorized implementation of Haversine applied on NumPy arrays

#### Timing vectorized implementation

In [55]:
# Vectorized implementation of Haversine applied on NumPy arrays
%timeit df['distance'] = haversine(40.671, -73.985,\
                         df['latitude'].values, df['longitude'].values)

212 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [56]:
%%timeit
# Convert pandas arrays to NumPy ndarrays
np_lat = df['latitude'].values
np_lon = df['longitude'].values

4.23 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### Profiling vectorized implementation

In [57]:
%lprun -f haversine df['distance'] = haversine(40.671, -73.985,\
                        df['latitude'].values, df['longitude'].values)

Timer unit: 1e-06 s

Total time: 0.001106 s
File: <ipython-input-49-ad000f5aaf6c>
Function: haversine at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def haversine(lat1, lon1, lat2, lon2):
     2         1          2.0      2.0      0.2      miles_constant = 3959
     3         1         44.0     44.0      4.0      lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
     4         1        112.0    112.0     10.1      dlat = lat2 - lat1 
     5         1         33.0     33.0      3.0      dlon = lon2 - lon1 
     6         1        475.0    475.0     42.9      a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
     7         1        434.0    434.0     39.2      c = 2 * np.arcsin(np.sqrt(a)) 
     8         1          5.0      5.0      0.5      mi = miles_constant * c
     9         1          1.0      1.0      0.1      return mi

## Cythonize that loop

#### Load the cython extension

In [58]:
%load_ext cython

#### Run unaltered Haversine through Cython

In [59]:
%%cython -a

# Haversine cythonized (no other edits)
import numpy as np
cpdef haversine_cy(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

#### Time it

In [60]:
%timeit df['distance'] =\
       df.apply(lambda row: haversine_cy(40.671, -73.985,\
                row['latitude'], row['longitude']), axis=1)

58.4 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Redefine Haversine with data types and C libraries

In [61]:
%%cython -a
# Haversine cythonized
from libc.math cimport sin, cos, acos, asin, sqrt

cdef deg2rad_cy(float deg):
    cdef float rad
    rad = 0.01745329252*deg
    return rad
    
cpdef haversine_cy_dtyped(float lat1, float lon1, float lat2, float lon2):
    cdef: 
        float dlon
        float dlat
        float a
        float c
        float mi
    
    lat1, lon1, lat2, lon2 = map(deg2rad_cy, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    mi = 3959 * c
    return mi


#### Time it

In [62]:
%timeit df['distance'] =\
df.apply(lambda row: haversine_cy_dtyped(40.671, -73.985,\
                              row['latitude'], row['longitude']), axis=1)

32.3 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
